In [8]:
def load_data(split_name='train', columns=['text', 'stars']):
    try:
        print(f"select [{', '.join(columns)}] columns from the {split_name} split")
        df = pd.read_csv(f'{split_name}.csv')
        df = df.loc[:,columns]
        print("succeed!")
        return df
    except:
        print("Failed, then try to ")
        print(f"select all columns from the {split_name} split")
        df = pd.read_csv(f'{split_name}.csv')
        return df

In [45]:
import pandas as pd
import numpy as np

In [29]:
matrix = load_data('matrix', "full")

select [f, u, l, l] columns from the matrix split
Failed, then try to 
select all columns from the matrix split


In [87]:
def cos_sim(userA, userB):
    userArt = subset.loc[subset['UserId'] == userA].to_numpy().flatten()[1:]
    userBrt = subset.loc[subset['UserId'] == userB].to_numpy().flatten()[1:]
    mask = ~(np.isnan(userArt) | np.isnan(userBrt))
    if mask.sum() < 4:
         return -2
    return np.dot(userArt[mask], userBrt[mask]) / (np.sqrt(np.dot(userArt[mask], userArt[mask])) * np.sqrt(np.dot(userBrt[mask], userBrt[mask])))

In [31]:
subset = matrix.iloc[509:, 0:100]

In [95]:
def find_most_sim_user(cur_user):
    max_sim = -2
    max_user = -1
    for user in subset['UserId'].to_numpy():
        if user == cur_user:
            continue
        sim = cos_sim(cur_user, user)
        if sim > max_sim:
            max_sim = sim
            max_user = user
    return max_user

In [121]:
def find_new_movie(cur_user, sim_user):
    userArt = subset.loc[subset['UserId'] == cur_user].to_numpy().flatten()[1:]
    userBrt = subset.loc[subset['UserId'] == sim_user].to_numpy().flatten()[1:]
    mask = np.isnan(userArt) & ~(np.isnan(userBrt))
    movieIds = np.argwhere(mask).flatten()+1
    np.random.shuffle(movieIds)
    return movies['title'] [movies['movieId'].isin(movieIds[:10])].to_numpy()

In [110]:
movieIds = find_new_movie(605,599)

movieIds[:10]

array([55, 99, 52, 26, 44, 39, 16, 42, 31, 19], dtype=int64)

In [112]:
movies = load_data('movies', ['movieId', 'title'])
movies.head()

select [movieId, title] columns from the movies split
succeed!


,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [128]:
find_new_movie(605,599)

array(['Grumpier Old Men (1995)', 'American President, The (1995)',
       'Dracula: Dead and Loving It (1995)', 'Powder (1995)',
       'Othello (1995)', 'Juror, The (1996)',
       'White Balloon, The (Badkonake sefid) (1995)',
       'Once Upon a Time... When We Were Colored (1995)',
       'Dunston Checks In (1996)'], dtype=object)

In [132]:
def gen_suggestion(cur_user):
    sim_user = find_most_sim_user(cur_user)
    return find_new_movie(cur_user, sim_user)

In [133]:
gen_suggestion(605)

array(['American President, The (1995)', 'Four Rooms (1995)',
       'Othello (1995)', 'Dangerous Minds (1995)',
       'Twelve Monkeys (a.k.a. 12 Monkeys) (1995)', 'Richard III (1995)',
       'French Twist (Gazon maudit) (1995)', 'Juror, The (1996)',
       "Things to Do in Denver When You're Dead (1995)",
       'Nick of Time (1995)'], dtype=object)